# TensorFlow Neural Network Model Building
This Jupyter Notebook is the product of Thomas Hymel. The contents pertain to one part of a Automatic Drum Transcription (ADT) project that I am working on to improve my data science and machine learning skills. This notebook in particular focuses on the neural network model building using Keras and TensorFlow. The data used for training the model will initially be the 25 song data set that has been created in other Jupyter Notebooks. The training data is in two matrices X (input) and Y (output labels) that will be loaded into this Notebook, along with a python dictionary that describes the X and Y data sets. 

#### Keras and TensorFlow
Keras is a deep learning API written in Python that runs on top of the machine learning platform TensorFlow. It allows users to quickly and easily build a model with layers as building blocks. I will be using Keras layers to build up a CNN, and perhaps a CRNN after that depending on the success (or expected lack of success) of the CNN. 

#### Considerations
This project falls under the "multi-label" classification model. This means that each example can be *multiple* classes of the available final classification options. Specifically, a bass drum event is independent from a snare drum event and from a hihat event and from a cymbal event and from a tom event, because a drummer, having multiple limbs, is able to simultaneously play multiple of these drum pieces. As such, the one-hot matrix Y can have multiple 1s in any given example row. 

Because of this, the final dense layer in the model **should not** be using the softmax ativation function. The softmax function chooses exactly one class and it is weighted by the presence of the other classes. Thus the activation function for the final layer needs to be **sigmoid function** because it properly projects the input to a probability between 0 and 1 without any consideration of or weighting from the other classes (using activation = 'sigmoid'). Additionally, the loss function used when the model is compiled should be loss='binary_crossentropy'. 

The current state of the X matrix can be thought of as the following: the X matrix is a gigantic image that is ~200 pixels tall and ~100,000 pixels wide. Each example, that is each column of pixels (200 pixels tall) is labeled with whatever is "in" it (by the Y matrix). However the examples are **ordered** because they are derived from a time sequence data set (songs), so there is information inherently "encoded" in the order of the examples themselves. I believe that the context of each example (the column of pixels around any given example) is useful to classify the current example, but not necessarily a huge amount of context. Local context, somewhere from 4-20 examples, I think would be most useful to help classify any specific example. Anything more than that I believe wouldn't actually help. 

In [43]:
# import relevant packages
import pandas as pd
import numpy as np
import random
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *

# get the X, Y, and note_dict data from the 25 song training set
fp = "C:\\Users\\Thomas\\Python Projects\\Drum-Tabber-Support-Data\\Saved-Output\\"
X_temp = np.load(fp + 'X_june29th.npy')
Y_all = np.load(fp + 'Y_june29th.npy')
with open(fp + 'note_dict_june29th.json') as json_file: 
    note_dict = json.load(json_file)
X_all = np.transpose(X_temp, (1,0,2))   # because of the way I had previously output the numpy array, should put the num_examples first. Code has been fixed in create_XY function

'\n# get the X, Y, and note_dict data from the 25 song training set\nfp = "C:\\Users\\Thomas\\Python Projects\\Drum-Tabber-Support-Data\\Saved-Output\\"\nX_temp = np.load(fp + \'X_june29th.npy\')\nY_all = np.load(fp + \'Y_june29th.npy\')\nwith open(fp + \'note_dict_june29th.json\') as json_file: \n    note_dict = json.load(json_file)\nX_all = np.transpose(X_temp, (1,0,2))   # because of the way I had previously output the numpy array, should put the num_examples first\n'

In [373]:
print("X_all.shape", X_all.shape)
print("Y_all.shape", Y_all.shape)
print(list(note_dict.keys()))
print(note_dict['X song index range'])
index_range_d = note_dict['X song index range']
print(int(0.15 * len(note_dict['X song index range'].keys())))
random_sample = sorted(random.sample(list(index_range_d.keys()), k = 4))
random_sample2 = sorted(random.sample([x for x in list(index_range_d.keys()) if x not in random_sample], k = 4))
print(random_sample)
print(random_sample2)
combined = [random_sample, random_sample2]
print(combined)
                         
print(index_range_d[random_sample[0]])
print(X_all[index_range_d[random_sample[0]][0]:index_range_d[random_sample[0]][1]][:][:].shape)

Xtrain_test, Ytrain_test, Xdev_test, Ydev_test, Xtest_test, Ytest_test = split_XY(X_all, Y_all, note_dict)

X_all.shape (199568, 200, 3)
Y_all.shape (199568, 7, 3)
['n_mels', 'sr', 'n_spectro_slices', 'include_LR', 'include_differential', 'X song order', 'X song length', 'X song index range', 'X spectro index order', 'Y column index order']
{'ancient_tombs': [0, 11523], 'best_of_me': [11524, 18031], 'boulevard_of_broken_dreams': [18032, 23787], 'cant_be_saved': [23788, 32223], 'face_down': [32224, 41459], 'family_tradition': [41460, 51247], 'fireworks_at_dawn': [51248, 53675], 'forever_at_last': [53676, 60819], 'four_years': [60820, 69351], 'garden_state': [69352, 79979], 'gunpowder': [79980, 92575], 'hair_of_the_dog': [92576, 100187], 'lungs_like_gallows': [100188, 108071], 'misery_business': [108072, 117667], 'mookies_last_christmas': [117668, 125407], 'planning_a_prison_break': [125408, 140571], 'rollercoaster': [140572, 147871], 'sow': [147872, 153779], 'sugar_were_going_down': [153780, 158263], 'surprise_surprise': [158264, 165747], 'thats_what_you_get': [165748, 173135], 'the_dark': [1

In [399]:
# print(note_dict['Y column index order'])
print(Xtrain_test.shape)
print(Ytrain_test.shape)
print(Xtrain_test.shape[0] + Xdev_test.shape[0] + Xtest_test.shape[0])
print(199568*3)


(469308, 200)
(469308, 7)
598704
598704


#### split_XY function
Although there are probably TensorFlow functions that automatically do the following, I am going to write a function that splits the X and Y data sets into training, development (dev) and tests sets. I am doing this because I am attempting to preserve the order of the examples, since they are still in chronological order. 

In [364]:
def split_XY(X,Y, note_dict, dev_split = 0.15, test_split = 0.15, sample_each_song = False):
    """
    Splits the full X and Y data set into a train, dev, and test split, based normally on the choice of random SONGS,
    unless the sample each song boolean is true. If true, the data set is split in such a way that it takes a sequential portion
    of each song, where the total portions are roughly each equal to the dev and test splits.
    
    Args:
        X [np.array]: full X set from the create_XY function (n_examples, n_mels_total, n_spectro_channels)
        Y [np.array]: full Y set from the create_XY function (n_examples, n_mels_total, n_spectro_channels)
        note_dict [dict]: dictionary created in create_XY used to describe the 
        dev_split [float]: fraction of the full X, Y set to put in the dev set
        test_split [float]: fraction of the full X, Y set to put in the test set (will not overlap with dev set)
        sample_each_song [bool]: Default = False. If False, will separate the dev and test by FULL SONG only. 
                                                  If True, will take a continuous duration sample from each song in the training set, based on the dev and test split fractions
        
    Returns:
        np.array: X_train
        np.array: Y_train
        np.array: X_dev
        np.array: Y_dev
        np.array: X_test
        np.array: Y_test
    """
    
    def unstack(matrix):
        """
        Helper function used to undo the spectro channel stacking and to place them in one large stack (finally treating them as individual examples)
        
        Args:
            matrix [np.array]: any np array, but expects it to have the dimensions of (n_examples, n_features/classes, n_channels)
        
        Returns:
            np.array: a 2 dimensional numpy array, either unstacked (in the include LR channels case) or simply not a 3D numpy array anymore (in the mono only case)
        """
        to_concat= []
        for index in range(matrix.shape[2]):  # assumes the third dimension describes the number of channels
            to_concat.append(matrix[:,:,index])
        return np.concatenate(to_concat, axis = 0)
        
    song_index_dict = note_dict['X song index range']  # gets the dict where { song_title : index_range_of_that_songs_examples ([index_start, index_end]) }
    songs = list(song_index_dict.keys())     # list of song title strings in entire data set
    n_songs = len(songs)     # number of songs in the entire data set
    
    if not sample_each_song: # in the case where each song is NOT sampled, and instead entire songs are chosen for the dev and test sets
        n_songs_dev = int(dev_split*n_songs)  # int number of songs in dev set
        n_songs_test = int(test_split*n_songs)  # int number of songs in test set
        songs_dev = sorted(random.sample(songs, k = n_songs_dev))    # grab dev number of songs from the songs list, and sorts alphabetically
        songs_test = sorted(random.sample([x for x in songs if x not in songs_dev], k = n_songs_test))  # grab test number of songs from the songs list, but not including the dev list, and sorts alphabetically
        
        set_list = [songs_dev, songs_test]  # ha, set_list, like a band would play live a bunch of songs
        
        XdYdXtYt = [] # list of np.arrays that will correspond to, in order, X_dev, Y_dev, X_test, Y_test
        index_range_list = [] # to be 
        # get slices of X,Y that correspond to the dev set first time through, test set second time through
        for song_list in set_list:
            X_dt_list = []   # will be a list of np.arrays from X
            Y_dt_list = []   # will be a list of np.arrays from Y
            for song in song_list:
                index_range = song_index_dict[song]
                index_range_list.append(index_range)   # keep all the index ranges [start, end_inclusive] that are used in dev and test set
                X_song = X[ index_range[0]:index_range[1]+1 ][:][:]  # add 1 because the saved indices are INCLUSIVE and Python uses exclusive slicing
                Y_song = Y[ index_range[0]:index_range[1]+1 ][:][:]
                X_dt_list.append(X_song)
                Y_dt_list.append(Y_song)
            X_dt = np.concatenate(X_dt_list, axis = 0)
            Y_dt = np.concatenate(Y_dt_list, axis = 0)
            XdYdXtYt.append(X_dt)
            XdYdXtYt.append(Y_dt)

        # set the outputs equal to their proper sets from the list
        X_dev, Y_dev, X_test, Y_test = unstack(XdYdXtYt[0]), unstack(XdYdXtYt[1]), unstack(XdYdXtYt[2]), unstack(XdYdXtYt[3])
        
        # delete the indexes of the used songs from the X and Y full sets to create the X and Y training sets
        all_index = []
        for obj in index_range_list:
            all_index = all_index + list(range(obj[0], obj[1]+1))
        X_train = unstack(np.delete(X, all_index, axis=0))
        Y_train = unstack(np.delete(Y, all_index, axis=0))
        
    else: # in the case where each song IS sampled roughly equally according to the dev_split and test_split, still in chrono order in the songs and alphabetical by song title
        None
    
    return X_train, Y_train, X_dev, Y_dev, X_test, Y_test

In [395]:
def CNN_2018(n_classes, n_mels, n_spectro_context, include_differential):
    """
    Creates a Keras NN model for the processing of the data. 
    Note that this NN model is based on the 2018 "Towards Multi-Instrument Drum Transcription" paper's model
    
    Args:
        n_classes [int]: the total number of classes output by the model, as dictated by the the second dimension of the Y matrix
        n_mels [int]: the total number of mels used to create the spectrogram slices. Decided by the note_dict (made in create_XY function)
        n_spectro_context [int]: (should be greater than 3)
        include_differential [bool]: Did the data set include the first time differential. Decided by the note_dict
        
    Returns:
        Keras Model:
    """
    
    n_features = n_mels
    if include_differential:        
        n_features = n_features * 2   # double the number of features if first time differential was included
    input_shape = (n_features, n_spectro_context, 1)  # to use the Conv2D later on we need to say there is only 1 channel in the 
    
    x = Input(shape = input_shape, name = 'input', dtype = 'float32')  # returns a TensorFlow symbolic tensor object
    
    # normalization of the initial input data
    y = BatchNormalization(axis=1)(x)  # batch norm is normally done over the feature axis. Current shape here is: (None, features, context (time axis), channel=1)
    
    # 2 x convolutional layer (32 filter x  3x3)
    y = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(y)   # Conv2D  (num_filters, (kernal_size_dim1, kernal_size_dim2), strides = (1,1), padding = 'valid', ) 'Valid' padding has NO padding, so "image" will shrink. "Same" padding HAS padding to preserve the dimensions of the "image"
    y = BatchNormalization()(y)
    y = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(y)
    y = BatchNormalization()(y)
    
    # max pool (1x3)
    y = MaxPool2D(pool_size = (1,3))(y)
    
    # 2 x convolutional layer (64 filter x  3x3)
    y = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(y)   # Conv2D  (num_filters, (kernal_size_dim1, kernal_size_dim2), strides = (1,1), padding = 'valid', ) 'Valid' padding has NO padding, so "image" will shrink. "Same" padding HAS padding to preserve the dimensions of the "image"
    y = BatchNormalization()(y)
    y = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(y)
    y = BatchNormalization()(y)
    
    
     # max pool (1x3)
    y = MaxPool2D(pool_size = (1,3))(y)
    
    # 2 x dense (256)
    y = Flatten()(y)
    y = Dense(256, activation = 'relu')(y)
    y = Dense(256, activation = 'relu')(y)
    y = Dense(n_classes, activation = 'sigmoid')(y)
    
    return keras.Model(inputs = x, outputs = y)

In [389]:
testmodel = CNN_2018(7, 100, 32, True)
testmodel.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
testmodel.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 200, 32, 1)]      0         
_________________________________________________________________
batch_normalization_90 (Batc (None, 200, 32, 1)        800       
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 200, 32, 32)       320       
_________________________________________________________________
activation_91 (Activation)   (None, 200, 32, 32)       0         
_________________________________________________________________
batch_normalization_91 (Batc (None, 200, 32, 32)       128       
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 200, 32, 32)       9248      
_________________________________________________________________
activation_92 (Activation)   (None, 200, 32, 32)       0   

In [394]:
testmodel.fit(x= np.stack([Xtrain_test],axis=-1), y = Ytrain_test, epochs=1, shuffle=False)

ValueError: Error when checking input: expected input to have 4 dimensions, but got array with shape (469308, 200, 1)

In [424]:
def NoContextModel(n_classes, n_mels, include_differential):
    """
    Simple model to test the idea that each spectro slice, with no context at all, can be used to classify the drum events
    that occur within that spectro slice.
    
    Args:
        asdf
        
    Returns:
        Keras model
    
    """
    
    n_features = n_mels
    if include_differential:        
        n_features = n_features * 2   # double the number of features if first time differential was included
    input_shape = (n_features, 1)  # to use the Conv1D later on we need to say there is only 1 channel in the 
    
    x = Input(shape = input_shape, name = 'input', dtype = 'float32')  # returns a TensorFlow symbolic tensor object
    
    # normalization of the initial input data
    y = BatchNormalization(axis=1)(x)  # batch norm is normally done over the feature axis. Current shape here is: (None, features, context (time axis), channel=1)
    
    # Convolutional Layers
    y = Conv1D(4, n_features, padding = 'same', activation = 'relu')(y)   # Conv1D  (num_filters, (kernal_size_dim1, kernal_size_dim2), strides = (1,1), padding = 'valid', ) 'Valid' padding has NO padding, so "image" will shrink. "Same" padding HAS padding to preserve the dimensions of the "image"
    y = BatchNormalization()(y)
    y = Conv1D(2, n_features, padding = 'same', activation = 'relu')(y) 
    
    # Dense layers
    y = Flatten()(y)
    y = Dense(64, activation = 'relu')(y)
    y = BatchNormalization()(y)
    y = Dense(n_classes, activation = 'sigmoid')(y)
    
    return keras.Model(inputs = x, outputs = y)

In [425]:
testNCM = NoContextModel(7, 100, True)
testNCM.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy', keras.metrics.Recall()])
testNCM.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 200, 1)]          0         
_________________________________________________________________
batch_normalization_114 (Bat (None, 200, 1)            800       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 200, 4)            804       
_________________________________________________________________
batch_normalization_115 (Bat (None, 200, 4)            16        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 200, 2)            1602      
_________________________________________________________________
flatten_23 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 64)                256

In [426]:
# training some model
testNCM.fit(x=np.stack([Xtrain_test],axis=-1), y = Ytrain_test, epochs=1, batch_size= 32, shuffle=False)

Train on 469308 samples
469308/469308 [==============================] - 818s 2ms/sample - loss: 0.1082 - accuracy: 0.9668 - recall: 0.1839


In [427]:
dev_dict_test = testNCM.evaluate(x= np.stack([Xdev_test],axis=-1), y= Ydev_test, batch_size=32)
print(dev_dict_test)

64536/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.26440627245874126, 0.91096646, 0.20287421]


In [473]:
Yprediction = testNCM.predict(x= np.stack([Xtrain_test],axis=-1))

In [510]:
example_num = 3454
print(note_dict['Y column index order'])
print(Yprediction[example_num]>0.5)
print(Ytest_test[example_num])

{'0': 'tk_beat', '1': 'tk_downbeat', '2': 'BD_o', '3': 'SD_o', '4': 'HH_x', '5': 'at_o', '6': 'ac_x'}
[ True False False  True False False  True]
[0 0 0 0 0 0 0]


##### List of Useful Shortcuts

* Ctrl + shift + P = List of Shortcuts
* Enter (command mode) = Enter Edit Mode (enter cell to edit it)
* Esc (edit mode) = Enter Command Mode (exit cell)
* A = Create Cell above
* B = Create Cell below
* D,D = Delete Cell
* Shift + Enter = Run Cell (code or markdown)
* M = Change Cell to Markdown
* Y = Change Cell to Code
* Ctrl + Shift + Minus = Split Cell at Cursor